In [1]:
import numpy as np


class SimplexSolver(object):

    def __init__(self, c, A, b):

        d = b < 0
        if d.any():
            raise ValueError("System is infeasible at the origin!")
        self.A = np.array(A)
        self.c = c
        self.b = b
        
        m,n = A.shape
        self.m,self.n=m,n
        self.L = list(range(n,n+m)+range(n))
        
        self.initialize_tableau()
        
    def initialize_tableau(self):
        self.Abar = np.bmat([self.A,np.eye(self.m)])
        self.cbar = np.hstack((np.hstack(self.c),np.hstack(np.zeros((self.m,1)))))
        self.T = np.bmat([[np.reshape(np.array([0]),(1,1)),-1.*np.reshape(self.cbar,(1,len(self.cbar))),np.array([[1]])],[np.reshape(self.b,(len(self.b),1)),self.Abar,np.zeros((len(self.b),1))]])
        
    def find_pivot(self):
        col = 1
        row = 1
        while self.T[0,col] >= 0:
            col+=1
        best = np.inf
       
        if not (self.T[:,col]>0).any():
            raise ValueError("Unbounded problem!")
        for num, thing in enumerate(self.T[1:,col]):
            if mask[num+1]:
                if self.T[num+1,0]/float(thing)<best:
                    best = self.T[num+1,0]/float(thing)
                    row = num+1
        self.col = col
        self.row = row
    
    def pivot(self):
        ind1 = self.row-1
        ind2 = self.L.index(self.col-1)
        self.L[ind1],self.L[ind2] = self.L[ind2],self.L[ind1]
        self.T[self.row,:]/=float(self.T[self.row,self.col])
        for i in xrange(len(self.T)):
            if i == self.row:
                pass
            else:
                factor = self.T[i,self.col]/float(self.T[self.row,self.col])
                self.T[i,:] -= factor * self.T[self.row]
    
    def solve(self):
        finished = False
        while not finished:
            if (self.T[0,1:]>=0).all():
                finished = True
                break
            self.find_pivot()
            self.pivot()
        optimum = self.T[0,0]
        basic = {}
        for i in xrange(self.m):
            basic[self.L[i]] = self.T[i+1,0]
        non_basic={}
        for i in xrange(self.m,len(self.L)):
            non_basic[self.L[i]] = 0
        return optimum,basic,non_basic


